<a href="https://colab.research.google.com/github/andraroman12/PMP-2024/blob/main/Examen/ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:

import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]

data = pd.read_csv(file_name)

features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
scaler = StandardScaler()
data[features] = scaler.fit_transform(data[features])

X = data[features].values
clusters = 3

with pm.Model() as model:

    weights = pm.Dirichlet("weights", a=np.ones(clusters))
    means = pm.Normal("means", mu=0, sigma=1, shape=(clusters, len(features)))
    sds = pm.HalfNormal("sds", sigma=1, shape=(clusters, len(features)))

    likelihoods = []
    for i in range(len(features)):
        likelihoods.append(
            pm.NormalMixture(
                f"likelihood_{features[i]}",
                w=weights,
                mu=means[:, i],
                sigma=sds[:, i],
                observed=X[:, i],
            )
        )

    trace = pm.sample(2000, tune=1000, return_inferencedata=True, random_seed=42)

print("Rezumatul inferenței:")
summary = az.summary(trace, var_names=["weights", "means", "sds"], hdi_prob=0.94)
print(summary)

feature_importance = []
for i, feature in enumerate(features):
    variability = trace.posterior["means"].sel(dim_1=i).std().mean().values
    feature_importance.append((feature, variability))

feature_importance = sorted(feature_importance, key=lambda x: x[1], reverse=False)
print("\nCaracteristica care separă cel mai bine datele este:", feature_importance[0][0])

az.plot_trace(trace, var_names=["weights"])
plt.show()

#Cea mai important caracteristica pentru separarea speciilor este cea cu cea mai mica variatie a mediilor, probabil  petal_length



Saving iris.csv to iris (4).csv


Output()

Output()

ValueError: Not enough samples to build a trace.